<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/introduction_to_tensorbord_callback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are on CoLaB with tensorflow version: {tf.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n...please load your drive...")
  COLAB = False
def time_fmt(t:float = 123.989)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}:{s:>05.2f}"
print(f"...time testing....time testing...time testing...\n>>>time elapse: {time_fmt()}")

Mounted at /content/drive
You are on CoLaB with tensorflow version: 2.4.1
...time testing....time testing...time testing...
>>>time elapse: 0: 02:03.00


In [ ]:
import tensorflow as tf
import numpy as np
import io, sys,time, os, datetime
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    name = 'cifar10',
    split = ['train', 'test'],
    shuffle_files = True,
    with_info = True,
    as_supervised = True
)
display(ds_info)
fig = tfds.show_examples(ds_train, ds_info, rows = 8, cols = 8)

In [ ]:
#pre-process the images:
def normalize_imgs(image, label):
  return (tf.cast(image, tf.float32)/255.0, label)

In [ ]:
def image_aug(image, label):
  if tf.random.uniform((), minval = 0, maxval = 1)<0.1: #Turn 10% of the images into gray scale
    image = tf.tile(tf.image.rgb_to_grayscale(images = image), [1,1,3])
  image = tf.image.random_brightness(image, max_delta = 0.1)
  image = tf.image.random_flip_up_down(image)
  return image, label
  

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
ds_train = ds_train.map(normalize_imgs, num_parallel_calls = AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.map(image_aug)
ds_train = ds_train.batch(batch_size = BATCH_SIZE, drop_remainder = True)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_imgs, num_parallel_calls = AUTOTUNE)
ds_test = ds_test.batch(batch_size = BATCH_SIZE, drop_remainder = True)
ds_test = ds_test.prefetch(ds_info.splits['train'].num_examples)


In [ ]:
classes = ["Airplane","Autmobile","Bird","Cat","Deer", "Dog", "Frog","Horse","Ship","Truck"]

In [ ]:
def cnn():
  inputs = tf.keras.Input(shape = (32,32,3))
  x = tf.keras.layers.Conv2D(filters = 64,
                             kernel_size = (3,3), 
                             activation = 'relu', 
                             padding = 'same', strides = (1,1),
                             kernel_initializer = 'random_normal', name = 'conv_1')(inputs)
  x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),
                             activation = 'relu',padding = 'same',
                             strides = (1,1), name = 'conv_2',
                             kernel_initializer = 'random_normal')(x)
  x = tf.keras.layers.MaxPooling2D(pool_size = (2,2), name = 'max_pool1')(x)
  x = tf.keras.layers.BatchNormalization(name = 'batch_norm_1')(x)
  x = tf.keras.layers.Conv2D(filters = 512, kernel_size = (3,3), activation = 'relu',
                             padding = 'same', strides = (1,1),
                             kernel_initializer = 'random_normal',
                             name = 'conv_3')(x)
  x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu',
                             padding = 'same', strides = (1,1), 
                             kernel_initializer = 'random_normal',
                             name = 'conv_4')(x)
  x = tf.keras.layers.MaxPooling2D(pool_size = (2,2), name = 'max_pool2')(x)
  x = tf.keras.layers.BatchNormalization(name = 'batch_norm_2')(x)
  x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),
                             activation = 'relu',padding = 'same',
                             strides = (1,1),kernel_initializer = 'random_normal',
                             name = 'conv_5')(x)
  x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', 
                             activation = 'relu', name = 'conv_6', strides = (1,1),
                             kernel_initializer = 'random_normal')(x)
  x = tf.keras.layers.Flatten(name = 'flatten')(x)
  x = tf.keras.layers.Dense(units = 128, activation = 'relu', name = 'dense1')(x)
  x = tf.keras.layers.Dropout(rate = 0.25, name = 'dropout')(x)
  x = tf.keras.layers.Dense(units = 64, activation = 'relu', name = 'dense2')(x)
  outputs = tf.keras.layers.Dense(units = 10, activation = 'softmax', name = 'outputs')(x)
  model = tf.keras.Model(inputs = inputs, outputs = outputs, name = 'cnn_simple')
  return model

                

In [ ]:
model = cnn()

In [ ]:
model.summary()
tf.keras.utils.plot_model(model = model, to_file = 'simple_cnn.png', show_shapes = True)

In [ ]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'Adam',
    metrics = ['sparse_categorical_accuracy']
)

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
callback_tfb = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
tic = time.time()
model.fit(ds_train, validation_data = ds_test, epochs = 10, callbacks = [callbacks_tfb], verbose = 2)
toc = time.time()
print(f"\ntotal time for training and validation is: {time_fmt(toc - tic)}")

In [ ]:
def cnn():
  inputs = tf.keras.Input(shape = (32,32,3))
  x = tf.keras.layers.Conv2D(filters = 64,
                             kernel_size = (3,3), 
                             activation = 'relu', 
                             padding = 'same', strides = (1,1),
                             kernel_initializer = 'random_normal', name = 'conv_1')(inputs)
  x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),
                             activation = 'relu',padding = 'same',
                             strides = (1,1), name = 'conv_2',
                             kernel_initializer = 'random_normal')(x)
  x = tf.keras.layers.MaxPooling2D(pool_size = (2,2), name = 'max_pool1')(x)
  x = tf.keras.layers.BatchNormalization(name = 'batch_norm_1')(x)
  x = tf.keras.layers.Conv2D(filters = 512, kernel_size = (3,3), activation = 'relu',
                             padding = 'same', strides = (1,1),
                             kernel_initializer = 'random_normal',
                             name = 'conv_3')(x)
  x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu',
                             padding = 'same', strides = (1,1), 
                             kernel_initializer = 'random_normal',
                             name = 'conv_4')(x)
  x = tf.keras.layers.MaxPooling2D(pool_size = (2,2), name = 'max_pool2')(x)
  x = tf.keras.layers.BatchNormalization(name = 'batch_norm_2')(x)
  x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),
                             activation = 'relu',padding = 'same',
                             strides = (1,1),kernel_initializer = 'random_normal',
                             name = 'conv_5')(x)
  x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', 
                             activation = 'relu', name = 'conv_6', strides = (1,1),
                             kernel_initializer = 'random_normal')(x)
  x = tf.keras.layers.Flatten(name = 'flatten')(x)
  x = tf.keras.layers.Dense(units = 128, activation = 'relu', name = 'dense1')(x)
  x = tf.keras.layers.Dropout(rate = 0.15, name = 'dropout')(x)
  x = tf.keras.layers.Dense(units = 64, activation = 'relu', name = 'dense2')(x)
  outputs = tf.keras.layers.Dense(units = 10, activation = 'softmax', name = 'outputs')(x)
  model = tf.keras.Model(inputs = inputs, outputs = outputs, name = 'cnn_simple')
  return model

In [ ]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)
optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
train_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_metric = tf.keras.metrics.SparseCategoricalAccuracy()
train_writer = tf.summary.create_file_writer("/logs/train/")
test_writer = tf.summary.create_file_writer("/logs/test/")

In [ ]:
train_step = test_step = 0
epochs = 10
for lr in [1e-1,1e-2,1e-3,1e-4,1e-5]:
  train_writer = tf.summary.create_file_writer("/logs/train" + str(lr))
  test_writer = tf.summary.create_file_writer("/logs/test/" + str(lr))
  model = cnn()
  optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr)
  for epoch in range(epochs):
    print(f"the start of epoch: {epoch}\n>>>training\tplease wait...")
    for (step, (x_train_batch, y_train_batch)) in enumerate(ds_train):
      with tf.GradientTape() as tape:
        preds = model(x_train_batch, training = True)
        train_loss = loss_obj(y_train_batch, preds)
      grads = tape.gradient(train_loss, model.trainable_weights)
      optimizer.apply_gradients(zip(grads, model.trainable_weights))
      train_metric.update_state(y_train_batch, preds)
      train_acc = train_metric.result()
      with train_writer.as_default():
        tf.summary.scalar('Loss', train_loss, step = train_step)
        tf.summary.scalar('Accuracy', train_acc, step = train_step)
        train_step+=1
      train_metric.reset_states()
    for (step, (x_val_batch, y_val_batch)) in enumerate(ds_test):
      preds = model(x_val_batch, training = False)
      val_loss = loss_obj(y_val_batch, preds)
      val_metric.update_state(y_val_batch, preds)
      val_acc = val_metric.result()
      with test_writer.as_default():
        tf.summary.scalar('Loss', val_loss, step = test_step)
        tf.summary.scalar('Accuracy', val_acc, step = test_step)
        test_step+=1
      val_metric.reset_states()




the start of epoch: 0
>>>training	please wait...
the start of epoch: 1
>>>training	please wait...
the start of epoch: 2
>>>training	please wait...
the start of epoch: 3
>>>training	please wait...
the start of epoch: 4
>>>training	please wait...
the start of epoch: 5
>>>training	please wait...
the start of epoch: 6
>>>training	please wait...
the start of epoch: 7
>>>training	please wait...
the start of epoch: 8
>>>training	please wait...
the start of epoch: 9
>>>training	please wait...
the start of epoch: 0
>>>training	please wait...
the start of epoch: 1
>>>training	please wait...
the start of epoch: 2
>>>training	please wait...
the start of epoch: 3
>>>training	please wait...
the start of epoch: 4
>>>training	please wait...
the start of epoch: 5
>>>training	please wait...
the start of epoch: 6
>>>training	please wait...
the start of epoch: 7
>>>training	please wait...
the start of epoch: 8
>>>training	please wait...
the start of epoch: 9
>>>training	please wait...
the start of epoch: 